In [26]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 2
BATCH_SIZE = 10
EPOCHS = 30
POSITION = "lying_down_left"
MODEL_NAME = f"position:{POSITION}_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"
DEV_SIZE = 7
TEST_SIZE = 7

### This code takes in a position, eg "sitting_or_standing", "lying_down_left" etc and trains a model for just classifying activities of that position

In [27]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split
import split_by_student


POSITIONS = {
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}

STANDING_OR_SITTING_ACTIVITIES = {
    "sitting&coughing",
    "sitting&hyperventilating",
    "sitting&normal_breathing",
    
    "standing&coughing",
    "standing&hyperventilating",
    "standing&normal_breathing",
    
    "sitting&talking",
    "sitting&eating",
    "sitting&singing",
    "sitting&laughing",
    
    "standing&talking",
    "standing&eating",
    "standing&singing",
    "standing&laughing",

}

STANDING_OR_SITTING_OUTCOMES= {
    "sitting_or_standing&normal_breathing",
    "sitting_or_standing&coughing",
    "sitting_or_standing&hyperventilating",
    "sitting_or_standing&other"
    


}


LYING_DOWN_LEFT_ACTIVITIES = {
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&talking",
    "lying_down_left&singing",
    "lying_down_left&laughing",
    "lying_down_left&normal_breathing"

}

LYING_DOWN_LEFT_OUTCOMES = {
    "lying_down_left&normal_breathing",
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    "lying_down_left&other"

}



LYING_DOWN_RIGHT_ACTIVITIES = {
    "lying_down_right&normal_breathing",
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_right&talking",
    "lying_down_right&singing",
    "lying_down_right&laughing"
}

LYING_DOWN_RIGHT_OUTCOMES = {
    "lying_down_right&normal_breathing",
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    "lying_down_right&other"
}

LYING_DOWN_BACK_ACTIVITIES = {
    "lying_down_back&normal_breathing",
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&talking",
    "lying_down_back&singing",
    "lying_down_back&laughing",
}

LYING_DOWN_BACK_OUTCOMES = {
    "lying_down_back&normal_breathing",
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    "lying_down_back&other",
}

LYING_DOWN_STOMACH_ACTIVITIES = {
    "lying_down_stomach&normal_breathing"
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&talking",
    "lying_down_stomach&singing",
    "lying_down_stomach&laughing",
}

LYING_DOWN_STOMACH_OUTCOMES= {
    "lying_down_stomach&normal_breathing"
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    "lying_down_stomach&other",
}

DATA_DIRECTORY = "./all_respeck"

position_activities = {"sitting_or_standing": STANDING_OR_SITTING_ACTIVITIES,
                       "lying_down_left": LYING_DOWN_LEFT_ACTIVITIES,
                       "lying_down_right": LYING_DOWN_RIGHT_ACTIVITIES,
                       "lying_down_back": LYING_DOWN_BACK_ACTIVITIES,
                       "lying_down_stomach": LYING_DOWN_STOMACH_ACTIVITIES}

position_outcomes = {"sitting_or_standing": STANDING_OR_SITTING_OUTCOMES,
                        "lying_down_left": LYING_DOWN_LEFT_OUTCOMES,
                        "lying_down_right": LYING_DOWN_RIGHT_OUTCOMES,
                        "lying_down_back": LYING_DOWN_BACK_OUTCOMES,
                        "lying_down_stomach": LYING_DOWN_STOMACH_OUTCOMES}


POSSIBLE_ACTIVITIES = position_activities[POSITION]
POSSIBLE_OUTCOMES = position_outcomes[POSITION]
LABEL_TO_INDEX = {label: idx for idx, label in enumerate(POSSIBLE_OUTCOMES)}



In [29]:
def generate_training_data(directory, sequence_length, overlap, file_names, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in POSSIBLE_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if csv_file in file_names:
                if gyro:
                    sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
                else:
                    sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

                # iterate through each generated sequence
                for sequence in sequences:
                    position = key.split("&")[0]
                    activity = key.split("&")[1]

                    if activity == "talking" or activity == "singing" or activity == "laughing" or activity == "eating":
                        activity = "other"

                    if position == "standing" or position == "sitting":
                        position = "sitting_or_standing"
                        
                    tagged_data.append((position + "&" + activity, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")

    return tagged_data

In [30]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        #layers.Conv1D(256, 3, activation='relu'),
        #layers.MaxPooling1D(2),
        #layers.Conv1D(512, 3, activation='relu'),
        #layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        #layers.Dense(256, activation='relu'),
        #layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if len(validation_data[0]) == 0:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
    else:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [ ]:
def create_sequence_label_lists(tagged_sequences):
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]
    sequences = np.array(sequences, dtype=np.float32)
    labels_encoded = [LABEL_TO_INDEX[label] for label in labels]
    labels = np.array(labels_encoded)

    return sequences, labels
    

def create_data_sets(dev_size, test_size):

    training_files, dev_files, test_files = split_by_student.split_data(students_in_dev_set= dev_size, students_in_test_set=test_size)

    tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
    tagged_dev_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=dev_files)
    tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=test_files)

    train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
    dev_data, dev_labels = create_sequence_label_lists(tagged_dev_sequences)
    test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

    #print(len(train_data), len(train_labels), len(dev_data), len(dev_labels), len(test_data), len(test_labels))

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [31]:

train_data, train_labels, dev_data, dev_labels, test_data, test_labels = create_data_sets(dev_size=DEV_SIZE, test_size=TEST_SIZE)


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
#model.save(f"models/models_for_presentation/lying_down_left_model.keras")

there are 5896 tagged sequences in the dataset
Epoch 1/50
472/472 [==============================] - 2s 3ms/step - loss: 1.2454 - accuracy: 0.5040 - val_loss: 1.2580 - val_accuracy: 0.4966
Epoch 2/50
472/472 [==============================] - 1s 3ms/step - loss: 1.2279 - accuracy: 0.5098 - val_loss: 1.2556 - val_accuracy: 0.4898
Epoch 3/50
472/472 [==============================] - 1s 3ms/step - loss: 1.2153 - accuracy: 0.5112 - val_loss: 1.2420 - val_accuracy: 0.4966
Epoch 4/50
472/472 [==============================] - 1s 3ms/step - loss: 1.2030 - accuracy: 0.5157 - val_loss: 1.2287 - val_accuracy: 0.5000
Epoch 5/50
472/472 [==============================] - 1s 3ms/step - loss: 1.1880 - accuracy: 0.5187 - val_loss: 1.1738 - val_accuracy: 0.5051
Epoch 6/50
472/472 [==============================] - 2s 3ms/step - loss: 1.0907 - accuracy: 0.5399 - val_loss: 1.1101 - val_accuracy: 0.5458
Epoch 7/50
472/472 [==============================] - 1s 3ms/step - loss: 1.0067 - accuracy: 0.5655 -

KeyboardInterrupt: 